### [Dogs vs. Cats Redux: Kernels Edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

Kaggleの，犬と猫を分類するタスクに取り組む．


このプログラムでは，
- [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)を継承したデータセット読み込み用クラスの定義
- [pretrainedmodels](https://github.com/Cadene/pretrained-models.pytorch)の学習済みモデルの利用
- マルチGPU実行
- [tensorboardX](https://github.com/lanpa/tensorboardX)を使い，TensorBoardでLoss・Accuracyを確認
- モデルの保存・読み込み

を行った．

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets

from pathlib import Path
import numpy as np

from PIL import Image
from tensorboardX import SummaryWriter
import pretrainedmodels

### 定数定義

In [2]:
root = 'dogs-vs-cats-redux-kernels-edition'
train_root = root + '/train'
test_root = root + '/test'

batch_size = 200

image_size = 224
resize_size = 256

lr = .01
momentum = .9

log_interval = 10

### データ読み込み
学習時のtransformでは，Data augmentationとしてランダムクロップとランダム水平フリップを行う．  
テスト時は，中央をクリップして画像サイズを合わせる．

PyTorchのデータセットを読み込む方法は主に3つあり，
- [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)を使う （公式に用意されたデータセットの場合）
- [torchvision.datasets.ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder)を使う （自分で用意したデータセットで，クラスごとにフォルダが分かれている場合）
- [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)を継承したクラスを使う （それ以外の場合，例えばクラスを示すCSVがある場合など）

今回は画像のファイル名が，犬か猫というクラスを表しているから，3つ目の方法をとる．  
与えられたのはTrainとTestだけで，Validationのデータが用意されていないので，Trainの10分の1をValidationとして確保する．

In [3]:
transform_train = transforms.Compose(
    [transforms.Resize(resize_size),
     transforms.RandomCrop(image_size),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])])

transform_test = transforms.Compose(
    [transforms.Resize(image_size),
     transforms.CenterCrop(image_size),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])])

class DogCatTrainSet(Dataset):
    def __init__(self, root, transform=None):
        self.root_dir = Path(root)
        self.transform = transform
        filelist_dog = sorted(self.root_dir.glob('dog.*.jpg'))[:11250]
        filelist_cat = sorted(self.root_dir.glob('cat.*.jpg'))[:11250]
        self.filelist = filelist_dog + filelist_cat
        
    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        image = Image.open(str(self.filelist[idx]))

        if self.transform:
            image = self.transform(image)
            
        label = int(self.filelist[idx].name.startswith('cat'))  # dog: 0, cat: 1

        return [image, label]

class DogCatValidationSet(Dataset):
    def __init__(self, root, transform=None):
        self.root_dir = Path(root)
        self.transform = transform
        filelist_dog = sorted(self.root_dir.glob('dog.*.jpg'))[11250:]
        filelist_cat = sorted(self.root_dir.glob('cat.*.jpg'))[11250:]
        self.filelist = filelist_dog + filelist_cat
        
    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        image = Image.open(str(self.filelist[idx]))

        if self.transform:
            image = self.transform(image)
            
        label = int(self.filelist[idx].name.startswith('cat'))  # dog: 0, cat: 1

        return [image, label]

class DogCatTestSet(Dataset):
    def __init__(self, root, transform=None):
        self.root_dir = Path(root)
        self.transform = transform
        self.filelist = [self.root_dir.joinpath('{}.jpg'.format(i)) for i in range(1, 12500+1)]
        
    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        image = Image.open(str(self.filelist[idx]))

        if self.transform:
            image = self.transform(image)
            
        return image

    
train_set = DogCatTrainSet(root=train_root, transform=transform_train)
validation_set = DogCatValidationSet(root=train_root, transform=transform_test)
test_set = DogCatTestSet(root=test_root, transform=transform_test)

train_loader = DataLoader(train_set, num_workers=2, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_set, num_workers=2, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, num_workers=2, batch_size=batch_size, shuffle=False)

### モデル定義
ImageNetで学習済みのResNeXtを使う．  
ResNeXtは公式で用意されていないので，[pretrainedmodels](https://github.com/Cadene/pretrained-models.pytorch)を使う．

ImageNetで学習済みのモデルをFine tuneするため，ネットワークの前半は学習しないようにrequires_gradをFalseにして，出力層の出力次元を1000から2に変更する．

[nn.DataParallel](https://pytorch.org/docs/stable/nn.html#torch.nn.DataParallel)でマルチGPU実行をする．

In [4]:
model = pretrainedmodels.resnext101_64x4d(num_classes=1000, pretrained='imagenet')

for p in model.features[:7].parameters():
    p.requires_grad = False

model.last_linear = nn.Linear(2048, 2)

model.cuda()
model = nn.DataParallel(model)

### Loss, Optimizer
ロスは[Cross Entropy](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)，
最適化は[Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam)を使う．

最適化関数のコンストラクタにmodel.parameters()を渡すと，requires_gradをFalseにしたことで学習できないパラメータがあるとエラーが出るので，学習する部分のパラメータのみ渡す．

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.module.features[7].parameters(), lr=lr)
optimizer.add_param_group({'params': model.module.last_linear.parameters()})

### tensorboardX
[tensorboardX](https://github.com/lanpa/tensorboardX)は，TensorBoardで見れる形式でデータを書き出してくれる．
学習時のLossや検証時のAccuracyを書き出すことで，TensorBoard上でグラフにして確認できる．

In [6]:
writer = SummaryWriter()

### 学習関数

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.cuda(), label.cuda()
        optimizer.zero_grad()
        
        out = model(data)
        
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        
        writer.add_scalar('Train/Loss', loss.item(), len(train_loader) * (epoch - 1) + batch_idx)
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{:5}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))  

### 検証関数

In [8]:
def validation(epoch):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in validation_loader:
            data, label = data.cuda(), label.cuda()
            out = model(data)
            val_loss += F.nll_loss(out, label, size_average=False).item()
            pred = out.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()
    
    accuracy = 100. * correct / len(validation_loader.dataset)
    writer.add_scalar('Validation/Accu', accuracy, epoch)
    
    val_loss /= len(validation_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

### テスト関数
テストデータを学習したモデルで推論し，Softmaxに通して犬である確率を書き出す．

In [9]:
def predict():
    model.eval()
    
    probs = np.array([])
    
    with torch.no_grad():
        for data in test_loader:
            data = data.cuda()
            out = model(data)
            softmax = F.softmax(out)
            probs = np.hstack([probs, softmax[:,0]])
    
    p = Path('submit')
    p.mkdir(exist_ok=True)
    with open(p.joinpath('dogsvscats.csv'), 'a') as f:
        f.write('id,label\n')
        submit = np.array(list(zip(np.arange(1, len(probs)+1), probs)))
        np.savetxt(f, submit, fmt='%d,%.8f')

### 学習
20エポック学習，
各エポックでテスト・モデルの保存をする．

In [10]:
epochs = 20

for i in range(epochs):
    train(i + 1)
    validation(i + 1)
    torch.save(model.state_dict(), 'models/model_epoch_' + str(i + 1))
    
writer.close()

Train Epoch: 1 [    0/22500 (0%)]	Loss: 0.664547
Train Epoch: 1 [ 2000/22500 (9%)]	Loss: 0.039342
Train Epoch: 1 [ 4000/22500 (18%)]	Loss: 0.049197
Train Epoch: 1 [ 6000/22500 (27%)]	Loss: 0.045172
Train Epoch: 1 [ 8000/22500 (35%)]	Loss: 0.036143
Train Epoch: 1 [10000/22500 (44%)]	Loss: 0.037868
Train Epoch: 1 [12000/22500 (53%)]	Loss: 0.196585
Train Epoch: 1 [14000/22500 (62%)]	Loss: 0.032759
Train Epoch: 1 [16000/22500 (71%)]	Loss: 0.224415
Train Epoch: 1 [18000/22500 (80%)]	Loss: 0.013995
Train Epoch: 1 [20000/22500 (88%)]	Loss: 0.033495
Train Epoch: 1 [22000/22500 (97%)]	Loss: 0.024425

Validation set: Average loss: -5.6625, Accuracy: 2477/2500 (99%)

Train Epoch: 2 [    0/22500 (0%)]	Loss: 0.034231
Train Epoch: 2 [ 2000/22500 (9%)]	Loss: 0.023230
Train Epoch: 2 [ 4000/22500 (18%)]	Loss: 0.005318
Train Epoch: 2 [ 6000/22500 (27%)]	Loss: 0.027988
Train Epoch: 2 [ 8000/22500 (35%)]	Loss: 0.021700
Train Epoch: 2 [10000/22500 (44%)]	Loss: 0.013547
Train Epoch: 2 [12000/22500 (53%)]	Lo

### テスト画像を推論
Validationの精度が最も高いモデルを読み込み，
testデータで推論して，Kaggleに提出するためのファイルを保存する．

In [11]:
model_name = 'models/model_epoch_19'
model.load_state_dict(torch.load(model_name))

predict()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


### 付録

![Train/Loss](trainloss.png)
![Val/Accu](valaccu.png)
![KaggleSubmit](kaggleres.png)